<a href="https://colab.research.google.com/github/katearzate/CSharp/blob/master/notebooks/Aguacate1980_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Obtención de los datos

El aguacate es uno de los principales productos agrícolas mexicanos es en la exportación agroalimentaria en México, convirtiendolo en el principal proveedor del mercado internacional. 
Lo que muchos no saben es que el epicentro de la producción se ha visto afectado con el crimen organizado con fines de lucro, de ahí paso su sobrenombre del aguacate de "oro verde" a "**diamante de sangre**".



Comenzaremos con obtener la colección de datos desde la Estadística de Producción Agrícola en http://infosiap.siap.gob.mx/gobmx/datosAbiertos.php, los cuales fueron guardados en el repositorio para mayor comodidad.


---



Importamos las librerias necesarias.

In [ ]:
import pandas as pd

Leemos los '.csv' y se concatenan en un solo dataframe. 

In [ ]:
%%capture
dfs = [pd.read_csv(f'https://raw.githubusercontent.com/mnaR99/narco_aguacate/main/data/raw/produccion/datosAbiertosGob/{anio}.csv', encoding='ISO-8859-1') for anio in range(1980,2020)]
df = pd.concat(dfs, sort='False', ignore_index='True')

# Análisis exploratorio de los datos

Vemos las columnas que lo componen.

In [ ]:
df.columns

Index(['Anio', 'Cosechada', 'Idcader', 'Idciclo', 'Idcultivo', 'Idddr',
       'Idestado', 'Idmodalidad', 'Idmunicipio', 'Idunidadmedida', 'Nomcader',
       'Nomcicloproductivo', 'Nomcultivo', 'Nomddr', 'Nomestado',
       'Nommodalidad', 'Nommunicipio', 'Nomunidad', 'Precio', 'Rendimiento',
       'Sembrada', 'Siniestrada', 'Unnamed: 18', 'Valorproduccion',
       'Volumenproduccion'],
      dtype='object')

 El tamaño del dataframe.

In [ ]:
df.shape

(689014, 25)

Un acercamiento para ver qué hay dentro de nuestro dataframe. 

In [ ]:
df.head(3)

,Anio,Idestado,Nomestado,Idddr,Nomddr,Idcader,Nomcader,Idmunicipio,Nommunicipio,Idciclo,Nomcicloproductivo,Idmodalidad,Nommodalidad,Idunidadmedida,Nomunidad,Idcultivo,Nomcultivo,Sembrada,Cosechada,Siniestrada,Volumenproduccion,Rendimiento,Precio,Valorproduccion,Unnamed: 18
0,2003.0,1.0,Aguascalientes,1.0,Aguascalientes,1.0,Aguascalientes,1.0,Aguascalientes,1.0,Otoño-Invierno,1.0,Riego,200201.0,Tonelada,5490000.0,Avena forrajera en verde,1062.0,1062.0,0.0,28529.0,26.86,323.4,9226278.6,NaN
1,2003.0,1.0,Aguascalientes,1.0,Aguascalientes,1.0,Aguascalientes,1.0,Aguascalientes,1.0,Otoño-Invierno,1.0,Riego,200201.0,Tonelada,5900000.0,Cebada forrajera en verde,2.0,2.0,0.0,54.0,27,250.0,13500.0,NaN
2,2003.0,1.0,Aguascalientes,1.0,Aguascalientes,1.0,Aguascalientes,1.0,Aguascalientes,1.0,Otoño-Invierno,1.0,Riego,200201.0,Tonelada,5980000.0,Centeno forrajero en verde,17.0,17.0,0.0,1020.0,60,350.0,357000.0,NaN


En México se cultivan diversas especies de semillas, pero solo nos interesa hacer el análisis del aguacate. Vamos a visualizar los cultivos que estan registrados, que son 463 cultivos.

In [ ]:
cultivos = pd.Series(df['Nomcultivo'].unique().tolist())
cultivos.sort_values(ascending=True).count()

463

In [ ]:
cultivos.tail(5)

459    Mano de león (manojo)
460        Orquídea (planta)
461         Helecho (manojo)
462         Pon-pon (gruesa)
463       Hortensia (planta)
dtype: object

# Corregir formato para la limpieza de datos

Nos interesa unicamente los registros de nombre de cultivo 'Aguacate'.

In [ ]:
dfA = df[df["Nomcultivo"] == "Aguacate"]
dfA.head(3)

,Anio,Cosechada,Idcader,Idciclo,Idcultivo,Idddr,Idestado,Idmodalidad,Idmunicipio,Idunidadmedida,Nomcader,Nomcicloproductivo,Nomcultivo,Nomddr,Nomestado,Nommodalidad,Nommunicipio,Nomunidad,Precio,Rendimiento,Sembrada,Siniestrada,Unnamed: 18,Valorproduccion,Volumenproduccion
166,1980.0,300.0,NaN,3.0,5060000.0,NaN,3.0,1.0,NaN,200201.0,NaN,Perennes,Aguacate,NaN,Baja California Sur,Riego,NaN,Tonelada,18.79,2.78,378.0,0.0,NaN,15652.0,833.0
204,1980.0,264.0,NaN,3.0,5060000.0,NaN,4.0,2.0,NaN,200201.0,NaN,Perennes,Aguacate,NaN,Campeche,Temporal,NaN,Tonelada,4.80,8,264.0,0.0,NaN,10138.0,2112.0
315,1980.0,239.0,NaN,3.0,5060000.0,NaN,6.0,1.0,NaN,200201.0,NaN,Perennes,Aguacate,NaN,Colima,Riego,NaN,Tonelada,8.25,3.31,866.0,0.0,NaN,6532.0,792.0


Las columnas 'Nomcicloproductivo', 'Nommodalidad', 'Nomunidad' y 'Nomcultivo' son descartados (al igual que la columna de su respectivo Id), donde los datos repetidos representan a:
*   Cultivo: Aguacate
*   Modalidad: Riego (modalidad hídrica)
*   Ciclo productivo: Perennes (continuo)
*   Unidad: Tonelada

In [ ]:
#dfNvo contiene todos las columnas que no contienen un dato repetido en todas las filas
dfNvo = dfA.loc[:,['Anio', 'Idestado', 'Nomestado', 'Idddr', 'Nomddr', 'Idcader',
       'Nomcader', 'Idmunicipio', 'Nommunicipio', 'Sembrada', 'Cosechada',
       'Siniestrada', 'Volumenproduccion', 'Rendimiento', 'Precio',
       'Valorproduccion']].sort_values('Anio',ascending=True)

dfNvo.head(3)

,Anio,Idestado,Nomestado,Idddr,Nomddr,Idcader,Nomcader,Idmunicipio,Nommunicipio,Sembrada,Cosechada,Siniestrada,Volumenproduccion,Rendimiento,Precio,Valorproduccion
166,1980.0,3.0,Baja California Sur,NaN,NaN,NaN,NaN,NaN,NaN,378.0,300.0,0.0,833.0,2.78,18.79,15652.0
1524,1980.0,20.0,Oaxaca,NaN,NaN,NaN,NaN,NaN,NaN,134.0,18.0,0.0,35.0,1.94,20.09,703.0
1531,1980.0,20.0,Oaxaca,NaN,NaN,NaN,NaN,NaN,NaN,1934.0,1875.0,0.0,36978.0,19.72,20.00,739560.0


Se observa si existen valores nulos y cuantos son por columna.



In [ ]:
dfNvo.isna().sum()

Anio                   0
Idestado               0
Nomestado              0
Idddr                981
Nomddr               981
Idcader              981
Nomcader             981
Idmunicipio          981
Nommunicipio         981
Sembrada               0
Cosechada              0
Siniestrada            0
Volumenproduccion      0
Rendimiento           56
Precio                56
Valorproduccion        0
dtype: int64



---

---




Con esto se puede deducir que para un análisis de producción por estado si es posible con los 40 años, pero por municipio no porque hace falta datos.


De los años 1980 a 2002 no se tienen las columnas 'Nomddr', 'Nomcader', 'Nommunicipio' y sus respectivos Id's. Se parten en 2 dataframes para el problema de los valores NaN, uno será para hacer el análisis de los 40 años por estado, y otro para análisis con todos los campos de 2002-2019.

#*   Transformación de datos para análisis por municipios (2003-2019)



In [ ]:
dfMun = dfNvo.dropna(how="all", subset=['Idddr'])
dfMun
dfMun.isna().sum()

Anio                  0
Idestado              0
Nomestado             0
Idddr                 0
Nomddr                0
Idcader               0
Nomcader              0
Idmunicipio           0
Nommunicipio          0
Sembrada              0
Cosechada             0
Siniestrada           0
Volumenproduccion     0
Rendimiento          56
Precio               56
Valorproduccion       0
dtype: int64

Los datos nulos de las columnas de 'Rendimiento' y 'Precio' se rellenaran con el último dato registrado para no alterar resultados.

In [ ]:
%%capture
dfMun['Rendimiento'] = dfMun['Rendimiento'].fillna(method='ffill')
dfMun['Precio'] = dfMun['Precio'].fillna(method='ffill')

In [ ]:
dfMun.isna().sum()

Anio                 0
Idestado             0
Nomestado            0
Idddr                0
Nomddr               0
Idcader              0
Nomcader             0
Idmunicipio          0
Nommunicipio         0
Sembrada             0
Cosechada            0
Siniestrada          0
Volumenproduccion    0
Rendimiento          0
Precio               0
Valorproduccion      0
dtype: int64

Checamos los tipos de variables que maneja cada columna.

In [ ]:
dfMun.dtypes

Anio                 float64
Idestado             float64
Nomestado             object
Idddr                float64
Nomddr                object
Idcader              float64
Nomcader              object
Idmunicipio          float64
Nommunicipio          object
Sembrada             float64
Cosechada            float64
Siniestrada          float64
Volumenproduccion    float64
Rendimiento           object
Precio               float64
Valorproduccion      float64
dtype: object

Convertimos al tipo de variable correcto.

In [ ]:
diccionario_casting = {
    'Anio': int,
    'Idestado': int,
    'Idddr': int,
    'Idcader': int,
    'Idmunicipio': int,
}
dfMun = dfMun.astype(diccionario_casting)
dfMun.dtypes

Anio                   int64
Idestado               int64
Nomestado             object
Idddr                  int64
Nomddr                object
Idcader                int64
Nomcader              object
Idmunicipio            int64
Nommunicipio          object
Sembrada             float64
Cosechada            float64
Siniestrada          float64
Volumenproduccion    float64
Rendimiento           object
Precio               float64
Valorproduccion      float64
dtype: object

Reseteamos el índice.

In [ ]:
dfMun.reset_index(drop=True).head(3)

,Anio,Idestado,Nomestado,Idddr,Nomddr,Idcader,Nomcader,Idmunicipio,Nommunicipio,Sembrada,Cosechada,Siniestrada,Volumenproduccion,Rendimiento,Precio,Valorproduccion
0,2003,20,Oaxaca,109,Cañada,1,Teotitlán de Flores Magón,163,San Jerónimo Tecóatl,12.0,12.0,0.0,42.0,3.5,2700.0,113400.0
1,2003,20,Oaxaca,109,Cañada,1,Teotitlán de Flores Magón,206,San Juan de Los Cués,6.0,6.0,0.0,21.0,3.5,3150.0,66150.0
2,2003,20,Oaxaca,109,Cañada,1,Teotitlán de Flores Magón,244,San Martín Toxpalan,11.0,11.0,0.0,66.0,6,5380.0,355080.0


¡Ya está listo! Solo falta guardarlo en un nuevo archivo.

In [ ]:
dfMun.to_csv(r'/dfMunicipios0319.csv', index = False)



---

---





#*   Transformación de datos para análisis por estados (1980-2019)

La columna de 'Nommunicipio' tambien se retira porque hay mas de 20 años donde no contiene su información, es por eso que este dataframe será solo para el análisis por estado.


Las columnas de 'Sembrada', 'Cosechada', 'Siniestrada', 'Rendimiento', 'Nomcader', 'Idcader', 'Nomddr' y 'Iddr', serán retiradas porque no son necesarias para hacer el análisis final del volumen y valor de producción de aguacate por municipio.
*   Nomddr: Distrito de Desarrollo Rural
*   Nomcader: Centro de Apoyo al Desarrollo Rural
*   Sembrada: hectáreas de superficie sembrada
*   Cosechada: hectáreas de superficie cosechada
*   Siniestrada: hectáreas de superficie dañada
*   Rendimiento: toneladas por hectárea


In [ ]:
dfEst = dfNvo.loc[:,['Anio', 'Idestado', 'Nomestado', 'Volumenproduccion', 'Precio', 'Valorproduccion']]
dfEst.head(3)

,Anio,Idestado,Nomestado,Volumenproduccion,Precio,Valorproduccion
166,1980.0,3.0,Baja California Sur,833.0,18.79,15652.0
1524,1980.0,20.0,Oaxaca,35.0,20.09,703.0
1531,1980.0,20.0,Oaxaca,36978.0,20.00,739560.0


Checamos por datos nulos.

In [ ]:
dfEst.isna().sum()

Anio                  0
Idestado              0
Nomestado             0
Volumenproduccion     0
Precio               56
Valorproduccion       0
dtype: int64

Llenamos los campos nulos en 'Precio' con el último registro.

In [ ]:
dfEst['Precio'] = dfEst['Precio'].fillna(method='ffill')
dfEst.isna().sum()

Anio                 0
Idestado             0
Nomestado            0
Volumenproduccion    0
Precio               0
Valorproduccion      0
dtype: int64

Arreglamos los tipos de variable para las columnas.

In [ ]:
dfEst.dtypes

Anio                 float64
Idestado             float64
Nomestado             object
Volumenproduccion    float64
Precio               float64
Valorproduccion      float64
dtype: object

In [ ]:
diccionary_casting = {
    'Anio': int,
    'Idestado': int,
}
dfEst = dfEst.astype(diccionary_casting)
dfEst.dtypes

Anio                   int64
Idestado               int64
Nomestado             object
Volumenproduccion    float64
Precio               float64
Valorproduccion      float64
dtype: object

In [ ]:
dfEst.reset_index(drop=True).head(3)

,Anio,Idestado,Nomestado,Volumenproduccion,Precio,Valorproduccion
0,1980,13,Hidalgo,3330.0,14.0,46620.0
1,1980,17,Morelos,1857.0,12.0,22284.0
2,1980,16,Michoacán,5800.0,11.0,63800.0


¡Hora de guardarlo y descargarlo!

In [ ]:
dfEst.to_csv(r'/dfEstados8019.csv', index = False)